In [154]:
import os

import matplotlib.pyplot as plt
import librosa
import numpy as np
import pandas as pd

from numpy import asarray
from numpy import savetxt
from os.path import join
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.decomposition import PCA
from sklearn.model_selection import train_test_split
import warnings
warnings.filterwarnings("ignore")

def convert_mel_one(sound):
    SR = 16000
    src, sr = librosa.load(sound, sr=SR, mono=True)
    len_second = 3.0 # 3 seconds
    src = src[:int(sr*len_second)] # crop first 3 seconds
    if len(src) < int(sr*len_second):
        N = int(sr*len_second) - len(src)
        src = np.pad(src, (0, N), 'constant')
    src_mel = librosa.feature.melspectrogram(y=src, sr=sr)
    return src_mel.flatten()


def save_csv_raw(sound_folder):
    files_list = os.listdir(sound_folder)    
    csv_file = np.zeros(12032)
    classifier = np.zeros(0)
    for i in files_list:
        if i.endswith('.wav'):
            converted = convert_mel_one(join(sound_folder,i))
            classifier_itr = i[:4]
            classifier = np.append(classifier,classifier_itr)
            csv_file = np.vstack((csv_file,converted))
            
    classifier = classifier[np.newaxis].T
    savetxt(sound_folder+'/raw.csv',csv_file,delimiter=",")
    return classifier
        
        

folder_name = '../data/vid/trainingSet/raw'
classifier = save_csv_raw(folder_name)




In [162]:
X_features = pd.read_csv('../data/vid/trainingSet/raw/raw.csv')
folder_name = '../data/vid/trainingSet/raw'
classifier = save_csv_raw(folder_name)


In [163]:
scaler = StandardScaler()
X_features = scaler.fit_transform(X_features)


In [171]:
pca = PCA(n_components=20)
pca.fit(X_features)
x_reduced = pca.transform(X_features)
full = pd.DataFrame(np.append(classifier, x_reduced,1))



In [172]:
full.to_csv (r'../data/vid/trainingSet/raw/full.csv', index = None, header=False)